In [10]:
import cv2
import numpy as np
from scipy.spatial.distance import euclidean
from imutils import perspective
from imutils import contours
import imutils

In [11]:
# Function to show array of images (intermediate results) with resizing capability
def show_images(images):
    for i, img in enumerate(images):
        window_name = "image_" + str(i)
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)  # Create a named window with resizing capability
        cv2.resizeWindow(window_name, 1024, 768)  # Resize the window to your desired size
        cv2.imshow(window_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
                                
# Capture an image from the webcam
def capture_image():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return None
    print("Press spacebar to capture the image, 'q' to quit")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            return None
        cv2.imshow('Webcam', frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord(' '):
            image_click = frame
            break
        elif key == ord('q'):
            return None
    cap.release()
    cv2.destroyAllWindows()
    return image_click

# Capture an image
image = capture_image()
if image is None:
    print("No image captured.")
    exit()

Press spacebar to capture the image, 'q' to quit


In [12]:
# Convert the captured image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7, 7), 0)

# Improved edge detection
edged = cv2.Canny(blur, 50, 150)
edged = cv2.dilate(edged, None, iterations=2)
edged = cv2.erode(edged, None, iterations=1)

# Find contours
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

# Sort contours from left to right as leftmost contour is reference object
(cnts, _) = contours.sort_contours(cnts)

# Remove contours which are not large enough
cnts = [x for x in cnts if cv2.contourArea(x) > 100]

# Reference object dimensions
ref_object = cnts[0]
box = cv2.minAreaRect(ref_object)
box = cv2.boxPoints(box)
box = np.array(box, dtype="int")
box = perspective.order_points(box)
(tl, tr, br, bl) = box
dist_in_pixel = euclidean(tl, tr)
dist_in_cm = 2  # Known width of the reference object in cm
pixel_per_cm = dist_in_pixel / dist_in_cm

In [13]:
# Draw remaining contours
for cnt in cnts:
    box = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(box)
    box = np.array(box, dtype="int")
    box = perspective.order_points(box)
    (tl, tr, br, bl) = box
    cv2.drawContours(image, [box.astype("int")], -1, (0, 0, 255), 2)
    mid_pt_horizontal = (tl[0] + int(abs(tr[0] - tl[0]) / 2), tl[1] + int(abs(tr[1] - tl[1]) / 2))
    mid_pt_verticle = (tr[0] + int(abs(tr[0] - br[0]) / 2), tr[1] + int(abs(tr[1] - br[1]) / 2))
    wid = euclidean(tl, tr) / pixel_per_cm
    ht = euclidean(tr, br) / pixel_per_cm
    cv2.putText(image, "{:.1f}cm".format(wid), (int(mid_pt_horizontal[0] - 15), int(mid_pt_horizontal[1] - 10)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
    cv2.putText(image, "{:.1f}cm".format(ht), (int(mid_pt_verticle[0] + 10), int(mid_pt_verticle[1])), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

# Display the result with resized window
show_images([image])